In [37]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os
from sklearn.utils.class_weight import compute_class_weight
import streamlit as st
from PIL import Image


In [39]:
# Dataset path
data_dir = "/Users/vijeethvj8/Downloads/Elevateme/Bone Break Classification"
img_size = (224, 224)
batch_size = 32

# Load training dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

# Load validation dataset
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

# Get class names
class_names = train_ds.class_names
print("Classes:", class_names)


Found 1129 files belonging to 10 classes.
Using 904 files for training.
Found 1129 files belonging to 10 classes.
Using 225 files for validation.
Classes: ['Avulsion fracture', 'Comminuted fracture', 'Fracture Dislocation', 'Greenstick fracture', 'Hairline Fracture', 'Impacted fracture', 'Longitudinal fracture', 'Oblique fracture', 'Pathological fracture', 'Spiral Fracture']


In [41]:
# ✅ Prefetch for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

# ✅ Calculate class weights
labels = []
for images, lbls in train_ds:
    labels.extend(np.argmax(lbls.numpy(), axis=1))

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.arange(len(class_names)),
    y=np.array(labels)
)
class_weights_dict = dict(enumerate(class_weights))
print("Class weights:", class_weights_dict)

# ✅ Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# ✅ Load EfficientNetB0 model
base_model = tf.keras.applications.EfficientNetB0(input_shape=img_size + (3,),
                                                   include_top=False,
                                                   weights='imagenet')
base_model.trainable = False

# ✅ Build the model
inputs = tf.keras.Input(shape=img_size + (3,))
x = data_augmentation(inputs)
x = tf.keras.applications.efficientnet.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(len(class_names), activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

# ✅ Compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# ✅ Model summary
model.summary()


Class weights: {0: 0.8776699029126214, 1: 0.7596638655462185, 2: 0.7232, 3: 0.8862745098039215, 4: 1.0761904761904761, 5: 1.3696969696969696, 6: 1.4580645161290322, 7: 1.3294117647058823, 8: 0.8692307692307693, 9: 1.2732394366197184}


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │        12,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,062,381 (15.50 MB)

 Trainable params: 12,810 (50.04 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [43]:
# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    class_weight=class_weights_dict,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
    ]
)


Epoch 1/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 25s 713ms/step - accuracy: 0.0918 - loss: 2.4012 - val_accuracy: 0.1556 - val_loss: 2.2599
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 24s 845ms/step - accuracy: 0.2182 - loss: 2.1445 - val_accuracy: 0.2133 - val_loss: 2.1619
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 24s 799ms/step - accuracy: 0.2736 - loss: 2.0194 - val_accuracy: 0.2489 - val_loss: 2.0959
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 21s 713ms/step - accuracy: 0.3394 - loss: 1.9612 - val_accuracy: 0.2889 - val_loss: 2.0322
Epoch 5/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 22s 738ms/step - accuracy: 0.3609 - loss: 1.9119 - val_accuracy: 0.3111 - val_loss: 2.0166
Epoch 6/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 700ms/step - accuracy: 0.3863 - loss: 1.7949 - val_accuracy: 0.2978 - val_loss: 2.0078
Epoch 7/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 21s 731ms/step - accuracy: 0.3545 - loss: 1.8237 - val_accuracy: 0.3111 - val_loss: 1.9964
Epoch 8/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 22s 751ms/step - accuracy: 0.4242 - loss: 1.7361 - val_accu

In [44]:
# Fine-tuning: Unfreeze last 30 layers
base_model.trainable = True
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Recompile with lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
fine_tune_history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    class_weight=class_weights_dict,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
    ]
)

Epoch 1/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 31s 923ms/step - accuracy: 0.4456 - loss: 1.5635 - val_accuracy: 0.3333 - val_loss: 1.8968
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 24s 822ms/step - accuracy: 0.4628 - loss: 1.5536 - val_accuracy: 0.3467 - val_loss: 1.9052
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 22s 768ms/step - accuracy: 0.4658 - loss: 1.5369 - val_accuracy: 0.3467 - val_loss: 1.9175
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 22s 760ms/step - accuracy: 0.4784 - loss: 1.5262 - val_accuracy: 0.3600 - val_loss: 1.9312


In [45]:
# Save the model
model.save("fracture_classifier_efficientnet.keras")


In [46]:
# ✅ Fine-tuning: unfreeze top layers of base model
base_model.trainable = True
for layer in base_model.layers[:-10]:
    layer.trainable = False

# ✅ Recompile with lower learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ✅ Fine-tune the model
fine_tune_history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    class_weight=class_weights_dict
)



Epoch 1/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 27s 766ms/step - accuracy: 0.4802 - loss: 1.5609 - val_accuracy: 0.3378 - val_loss: 1.9070
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 21s 716ms/step - accuracy: 0.4865 - loss: 1.5284 - val_accuracy: 0.3467 - val_loss: 1.9178
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 23s 808ms/step - accuracy: 0.4741 - loss: 1.5334 - val_accuracy: 0.3600 - val_loss: 1.9308
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 28s 872ms/step - accuracy: 0.5106 - loss: 1.5183 - val_accuracy: 0.3511 - val_loss: 1.9403
Epoch 5/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 21s 725ms/step - accuracy: 0.5020 - loss: 1.5410 - val_accuracy: 0.3333 - val_loss: 1.9494
Epoch 6/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 21s 707ms/step - accuracy: 0.4729 - loss: 1.4918 - val_accuracy: 0.3289 - val_loss: 1.9558
Epoch 7/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 21s 722ms/step - accuracy: 0.5139 - loss: 1.5076 - val_accuracy: 0.3333 - val_loss: 1.9609
Epoch 8/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 21s 727ms/step - accuracy: 0.4955 - loss: 1.4628 - val_accu

In [29]:
# ✅ Save fine-tuned model
model.save("fracture_classifier_finetuned.keras")